<a href="https://colab.research.google.com/github/KAVYANSHTYAGI/Food-Image-Classifier/blob/main/novel_architecture_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau , LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score,classification_report
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

base_dir_training = "/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/training"
class_labels_training = os.listdir(base_dir_training)
print(len(class_labels_training))

base_dir_testing = "/content/drive/MyDrive/Food Image Datasets/North Indian/main_final/testing"
class_labels_testing = os.listdir(base_dir_testing)
print(len(class_labels_testing))

In [ ]:
#training

data_training = []
count = 0

for label in class_labels_training:
    path = os.path.join(base_dir_training, label)
    print(path)
    for img in os.listdir(path):
        try:
            image = load_img(os.path.join(path,img), color_mode="rgb", target_size=(224,224))
            image = img_to_array(image)
            image /= 255.0
            data_training.append([image, count])

        except Exception as e:
            pass
    count = count + 1

#testing

data_testing = []
count = 0

for label in class_labels_testing:
    path = os.path.join(base_dir_testing, label)
    print(path)
    for img in os.listdir(path):
        try:
            image = load_img(os.path.join(path,img), color_mode="rgb", target_size=(224,224))
            image = img_to_array(image)
            image /= 255.0
            data_testing.append([image, count])

        except Exception as e:
            pass
    count = count + 1


In [ ]:
print(len(data_training))
print(len(data_testing))
print(count)
num_classes = count

In [ ]:
X_train,y_train = zip(*data_training)

x_train = np.array(X_train)
y_train = np.array(y_train)


X_test,y_test = zip(*data_testing)

x_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
y_test = to_categorical(y_test, num_classes=num_classes)
y_train = to_categorical(y_train, num_classes=num_classes)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'               # Fill missing pixels after transformations
)


val_test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow(x_train, y_train)
val_generator = val_test_datagen.flow(x_test, y_test)

model

In [ ]:
texture_input = layers.Input(shape=(224,224,3))
texture_x = Conv2D(32, 3, activation='relu')(texture_input)
texture_x = MaxPooling2D()(texture_x)

# Adding CBAM Attention Module (Spatial + Channel)
texture_x = cbam_block(texture_x)

texture_x = Conv2D(64, 3, activation='relu')(texture_x)
texture_x = GlobalAveragePooling2D()(texture_x)

texture_branch_model = Model(inputs=texture_input, outputs=texture_x)





color_input = layers.Input(shape=(224,224,3))
color_x = Conv2D(32, kernel_size=3, activation='relu')(color_input)
color_x = MaxPooling2D()(color_x)

# Channel Attention Module (SE Block)
color_gap = GlobalAveragePooling2D()(color_x)
color_attention = Dense(32//8, activation='relu')(color_x)
color_attention = Dense(32, activation='sigmoid')(color_attention)
color_x = Multiply()([color_x, color_attention])

color_x = GlobalAveragePooling2D()(color_x)





from tensorflow.keras.applications import vit
structure_input = layers.Input(shape=(224,224,3))

# ViT Small or Tiny pre-trained on ImageNet
vit_model = vit.ViT_B16(weights='imagenet', include_top=False, input_shape=(224,224,3))

structure_x = vit_model(structure_input)
structure_x = layers.GlobalAveragePooling1D()(structure_x)
structure_branch_output = layers.Dense(256, activation='relu')(structure_x)


In [ ]:
combined_features = layers.Concatenate()([texture_x, color_x, structure_branch])

# Attention-based fusion
attention_features = layers.MultiHeadAttention(num_heads=4, key_dim=64)(
    query=combined_features,
    value=combined_features,
    key=combined_features
)

attention_features = layers.GlobalAveragePooling1D()(attention_features)



x = Dropout(0.4)(attention_features)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(30, activation='softmax')(x)

final_model = Model(inputs=[texture_input, color_input, structure_input], outputs=output)


In [ ]:
history = model.fit(
    train_generator,
    epochs=40,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    shuffle=True)

# Plotting training and validation loss and accuracy
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
loss, accuracy, precision, recall, f1_score = model.evaluate(val_generator)
print(f'test Loss: {loss}, test Accuracy: {accuracy}')